In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

import random
random.seed(7)


def generator(X, Y, batch_size=32, train=True):
    while True:
        for offset in range(0, len(X), batch_size):
            X_batch = np.stack(X[offset:offset+batch_size], axis=0)
            Y_batch = np.stack(Y[offset:offset+batch_size], axis=0)
            
            Y_batch_ = np.empty((Y_batch.shape[0], 2,2,10))
            for m in range(Y_batch.shape[0]):
                for i in range(10):
                    Y_batch_[m, :,:,i] = [[np.sum(Y_batch[m, :5, :5, i]), np.sum(Y_batch[m, :5, 5:, i])], [np.sum(Y_batch[m, 5:, :5, i]), np.sum(Y_batch[m, 5:, 5:, i])]]
            Y_batch_[Y_batch_ > 0] = 1

            if train:
                Y_f = np.array([Y_batch_[e].flatten() for e in range(Y_batch_.shape[0])])
                yield (X_batch, Y_f)
            else:
                yield X_batch

structure_ids = []
for line in open('./structures lists/structures human.txt', 'r'):
    line = line.strip('\n')
    structure_ids.append(line)
# for line in open('./structures lists/structures ecoli.txt', 'r'):
#     line = line.strip('\n')
#     structure_ids.append(line)
structure_ids.remove('4pkd')
structure_ids.remove('1a9n')
structure_ids.remove('2adc')
random.shuffle(structure_ids)
print(len(structure_ids))

X_train = []
X_test_pos = []
X_test_neg = []
Y_train = []
Y_test_pos = []
Y_test_neg = []
num_aa_train = 0
num_aa_test_pos = 0
num_aa_test_neg = 0
num_train = int(len(structure_ids)*0.7)
for i, structure_id in enumerate(structure_ids):
    protein = np.load('../data/voxelized data 10x10x10/' + structure_id + '_protein.npy', mmap_mode='r')
    rna = np.load('../data/voxelized data 10x10x10/' + structure_id + '_rna_3D.npy', mmap_mode='r')
   
    na = 0
    pos = 0
    while (np.sum(rna[na]) > 100) and (na < len(rna)-1):
        pos +=1
        na +=1

    if i <= num_train:
        X_train.extend(protein[:pos, :, :, :, :3])
        Y_train.extend(rna[:pos])
        num_aa_train +=pos
#         if pos > len(rna)/2:
#             X_train.extend(protein[:, :, :, :, :3])
#             Y_train.extend(rna)
#             num_aa_train +=len(rna)
#         else:
#             X_train.extend(protein[:pos, :, :, :, :3])
#             X_train.extend(protein[-pos:, :, :, :, :3])
#             Y_train.extend(rna[:pos])
#             Y_train.extend(rna[-pos:])
#             num_aa_train +=2*pos
    else:
        X_test_pos.extend(protein[:pos, :, :, :, :3])
        Y_test_pos.extend(rna[:pos])
        X_test_neg.extend(protein[pos:, :, :, :, :3])
        Y_test_neg.extend(rna[pos:])
        num_aa_test_pos +=pos
        num_aa_test_neg +=(len(rna)-pos)

#         if pos > len(rna)/2:
#             X_test.extend(protein[:, :, :, :, :3])
#             Y_test.extend(rna)
#             num_aa_test +=len(rna)
#         else:
#             X_test.extend(protein[:pos, :, :, :, :3])
#             X_test.extend(protein[-pos:, :, :, :, :3])
#             Y_test.extend(rna[:pos])
#             Y_test.extend(rna[-pos:])
#             num_aa_test +=2*pos



   
n_steps_train = int(num_aa_train/400) 


generator_train = generator(X_train, Y_train, 400, True)

ins = tf.keras.layers.Input((10, 10, 10, 3))
con1 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same', activation='relu')(ins)
con2 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con1)
con3 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con2)
maxp1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con3)
con4 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp1)
con5 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con4)
con6 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con5)
maxp2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con6)
con7 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp2)
con8 = tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3), padding='same', activation='relu')(con7)
con9 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='same', activation='relu')(con8)
maxp3 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con9)
batch = tf.keras.layers.BatchNormalization()(maxp3)
flat = tf.keras.layers.Flatten()(batch)
dens2 = tf.keras.layers.Dense(units=256, activation='relu')(flat)
drop2 = tf.keras.layers.Dropout(0.6)(dens2)
outs = tf.keras.layers.Dense(units=40, activation='sigmoid')(drop2)
model = tf.keras.models.Model(inputs=ins, outputs=outs)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy', 'mse'])

model.summary()

# checkpoint
# filepath="weights_best.hdf5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

# model.fit(X_train, Y_train_f, validation_split=0.33, epochs=1, batch_size=200, callbacks=callbacks_list, verbose=0)
model.fit_generator(generator_train, steps_per_epoch=n_steps_train, epochs=500, callbacks=None, verbose=1)

def predict(X_test, Y_test, num_aa_test):
    Y_test = np.stack(Y_test, axis=0)
    Y_test_ = np.empty((len(Y_test), 2,2,10))
    for m in range(len(Y_test)):
        for i in range(10):
            Y_test_[m, :,:,i] = [[np.sum(Y_test[m, :5, :5, i]), np.sum(Y_test[m, :5, 5:, i])], [np.sum(Y_test[m, 5:, :5, i]), np.sum(Y_test[m, 5:, 5:, i])]]
    Y_test_[Y_test_ > 0] = 1

    n_steps_test = int(num_aa_test/400)
    generator_test = generator(X_test, Y_test, 400, False)

    # model_best = model
    # model_best.load_weights("weights_best.hdf5")
    # print(model.evaluate(X_test, Y_test, verbose=0, batch_size=100))
    # model_best.save('model_cnn_15_2.h5')
    # Y_pred = model_best.predict(X_test, batch_size=200)
    Y_pred = model.predict_generator(generator_test, steps=n_steps_test)
    print(Y_pred.shape)
    Y_pred_ = np.array([Y_pred[i].reshape((2,2,10)) for i in range(Y_pred.shape[0])])

    #CNN
    Y_pred_prob = Y_pred_
    Y_pred_[Y_pred_ >= 0.5] = 1
    Y_pred_[Y_pred_ < 0.5] = 0

    print(Y_pred_.shape)
    print(Y_test_.shape)
    Y_test_ = Y_test_[:Y_pred_.shape[0]]

    print('CNN: \n')
    for i in range(10):
        confusion_matrix = [sklearn.metrics.confusion_matrix(Y_test_[:,l , c, i], Y_pred_[:,l , c, i]) for l in range(2) for c in range(2)]
        accuracy = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix]
        auc = [sklearn.metrics.roc_auc_score(Y_test_[:,l , c, i], Y_pred_prob[:,l , c, i]) for l in range(2) for c in range(2)]

        print(f'level {i}')
        for q in range(len(confusion_matrix)):
            print(confusion_matrix[q], np.round(accuracy[q], 2), np.round(auc[q], 2))

    # baseline model
    # predict all zeros; at least 50% correct predictions because there are 1/2 of negative examples
    Y_pred_base = np.zeros(Y_test_.shape)

    # po = np.sum(Y_train, axis=0)/Y_train.shape[0]
    # po[po >= 0.5] = 1
    # po[po < 0.5] = 0
    # Y_pred_base = np.tile(po, (Y_test.shape[0],1))

    print(Y_pred_base.shape)
    print(f'\n BASELINE MODEL: \n')
    for i in range(10):
        confusion_matrix_base = [sklearn.metrics.confusion_matrix(Y_test_[:,l, c, i], Y_pred_base[:,l, c, i]) for l in range(2) for c in range(2)]
        accuracy_base = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_base]
        auc_base = [sklearn.metrics.roc_auc_score(Y_test_[:,l, c, i], Y_pred_base[:,l, c, i]) for l in range(2) for c in range(2)]

        print(f'level {i}')
        for q in range(len(confusion_matrix_base)):
            print(confusion_matrix_base[q], np.round(accuracy_base[q], 2), np.round(auc_base[q], 2))

    #random model
    Y_pred_random = np.random.random(Y_test_.shape)
    Y_pred_random[Y_pred_random >= 0.5] = 1
    Y_pred_random[Y_pred_random < 0.5] = 0

    print(f'\n RANDOM MODEL: \n')
    for i in range(10):
        confusion_matrix_random = [sklearn.metrics.confusion_matrix(Y_test_[:,l, c, i], Y_pred_random[:,l, c, i]) for l in range(2) for c in range(2)]
        accuracy_random = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_random]
        auc_random = [sklearn.metrics.roc_auc_score(Y_test_[:,l, c, i], Y_pred_random[:,l, c, i]) for l in range(2) for c in range(2)]

        print(f'level {i}')
        for q in range(len(confusion_matrix_random)):
            print(confusion_matrix_random[q], np.round(accuracy_random[q], 2), np.round(auc_random[q], 2))
            
            
predict(X_test_pos, Y_test_pos, num_aa_test_pos)
print('\n')
predict(X_test_neg, Y_test_neg, num_aa_test_neg)

230
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10, 10, 10, 3)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 10, 10, 10, 64)    5248      
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 10, 10, 10, 32)    55328     
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 10, 10, 10, 32)    27680     
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 5, 5, 5, 32)       0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 5, 5, 5, 32)       27680     
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 5, 5, 5, 16)       13840     
______

55/55 [==============================] - 7s 130ms/step - loss: 0.6371 - acc: 0.6544 - mean_squared_error: 0.2229
Epoch 44/500
55/55 [==============================] - 7s 124ms/step - loss: 0.6353 - acc: 0.6565 - mean_squared_error: 0.2220
Epoch 45/500
55/55 [==============================] - 7s 118ms/step - loss: 0.6350 - acc: 0.6550 - mean_squared_error: 0.2219
Epoch 46/500
55/55 [==============================] - 7s 132ms/step - loss: 0.6336 - acc: 0.6570 - mean_squared_error: 0.2213
Epoch 47/500
55/55 [==============================] - 7s 128ms/step - loss: 0.6314 - acc: 0.6609 - mean_squared_error: 0.2203
Epoch 48/500
55/55 [==============================] - 7s 127ms/step - loss: 0.6300 - acc: 0.6631 - mean_squared_error: 0.2196
Epoch 49/500
55/55 [==============================] - 7s 129ms/step - loss: 0.6287 - acc: 0.6634 - mean_squared_error: 0.2190
Epoch 50/500
55/55 [==============================] - 7s 127ms/step - loss: 0.6268 - acc: 0.6658 - mean_squared_error: 0.2181
Epoch

55/55 [==============================] - 7s 129ms/step - loss: 0.5705 - acc: 0.6974 - mean_squared_error: 0.1941
Epoch 109/500
55/55 [==============================] - 6s 118ms/step - loss: 0.5685 - acc: 0.6989 - mean_squared_error: 0.1932
Epoch 110/500
55/55 [==============================] - 6s 117ms/step - loss: 0.5659 - acc: 0.7032 - mean_squared_error: 0.1920
Epoch 111/500
55/55 [==============================] - 7s 128ms/step - loss: 0.5678 - acc: 0.6998 - mean_squared_error: 0.19292s - loss: 0.5700 - acc: 0.6
Epoch 112/500
55/55 [==============================] - 7s 119ms/step - loss: 0.5658 - acc: 0.7013 - mean_squared_error: 0.1921
Epoch 113/500
55/55 [==============================] - 7s 121ms/step - loss: 0.5666 - acc: 0.6996 - mean_squared_error: 0.1925
Epoch 114/500
55/55 [==============================] - 7s 124ms/step - loss: 0.5650 - acc: 0.7015 - mean_squared_error: 0.1918
Epoch 115/500
55/55 [==============================] - 7s 126ms/step - loss: 0.5623 - acc: 0.7048

55/55 [==============================] - 7s 124ms/step - loss: 0.5311 - acc: 0.7223 - mean_squared_error: 0.1787
Epoch 173/500
55/55 [==============================] - 7s 136ms/step - loss: 0.5347 - acc: 0.7175 - mean_squared_error: 0.1804
Epoch 174/500
55/55 [==============================] - 7s 130ms/step - loss: 0.5342 - acc: 0.7184 - mean_squared_error: 0.1802
Epoch 175/500
55/55 [==============================] - 7s 124ms/step - loss: 0.5324 - acc: 0.7198 - mean_squared_error: 0.1794
Epoch 176/500
55/55 [==============================] - 7s 119ms/step - loss: 0.5302 - acc: 0.7207 - mean_squared_error: 0.1786
Epoch 177/500
55/55 [==============================] - 7s 118ms/step - loss: 0.5303 - acc: 0.7215 - mean_squared_error: 0.1786
Epoch 178/500
55/55 [==============================] - 7s 128ms/step - loss: 0.5309 - acc: 0.7208 - mean_squared_error: 0.1789
Epoch 179/500
55/55 [==============================] - 7s 130ms/step - loss: 0.5296 - acc: 0.7218 - mean_squared_error: 0.178

55/55 [==============================] - 6s 117ms/step - loss: 0.5078 - acc: 0.7346 - mean_squared_error: 0.1701
Epoch 236/500
55/55 [==============================] - 7s 122ms/step - loss: 0.5097 - acc: 0.7330 - mean_squared_error: 0.1709
Epoch 237/500
55/55 [==============================] - 8s 137ms/step - loss: 0.5079 - acc: 0.7344 - mean_squared_error: 0.1701
Epoch 238/500
55/55 [==============================] - 7s 119ms/step - loss: 0.5092 - acc: 0.7329 - mean_squared_error: 0.1707
Epoch 239/500
55/55 [==============================] - 7s 128ms/step - loss: 0.5086 - acc: 0.7335 - mean_squared_error: 0.1705
Epoch 240/500
55/55 [==============================] - 6s 118ms/step - loss: 0.5049 - acc: 0.7360 - mean_squared_error: 0.1691
Epoch 241/500
55/55 [==============================] - 7s 129ms/step - loss: 0.5071 - acc: 0.7345 - mean_squared_error: 0.1699
Epoch 242/500
55/55 [==============================] - 7s 133ms/step - loss: 0.5057 - acc: 0.7354 - mean_squared_error: 0.169

55/55 [==============================] - 6s 114ms/step - loss: 0.4921 - acc: 0.7427 - mean_squared_error: 0.1643
Epoch 300/500
55/55 [==============================] - 7s 122ms/step - loss: 0.4902 - acc: 0.7444 - mean_squared_error: 0.1636
Epoch 301/500
55/55 [==============================] - 7s 121ms/step - loss: 0.4875 - acc: 0.7469 - mean_squared_error: 0.1625
Epoch 302/500
55/55 [==============================] - 7s 127ms/step - loss: 0.4917 - acc: 0.7438 - mean_squared_error: 0.1641
Epoch 303/500
55/55 [==============================] - 7s 119ms/step - loss: 0.4900 - acc: 0.7446 - mean_squared_error: 0.1635
Epoch 304/500
55/55 [==============================] - 7s 119ms/step - loss: 0.4916 - acc: 0.7439 - mean_squared_error: 0.1641
Epoch 305/500
55/55 [==============================] - 7s 125ms/step - loss: 0.4909 - acc: 0.7441 - mean_squared_error: 0.1638
Epoch 306/500
55/55 [==============================] - 7s 120ms/step - loss: 0.4899 - acc: 0.7448 - mean_squared_error: 0.163

55/55 [==============================] - 7s 120ms/step - loss: 0.4772 - acc: 0.7539 - mean_squared_error: 0.1586
Epoch 364/500
55/55 [==============================] - 7s 119ms/step - loss: 0.4763 - acc: 0.7546 - mean_squared_error: 0.1583
Epoch 365/500
55/55 [==============================] - 6s 116ms/step - loss: 0.4757 - acc: 0.7545 - mean_squared_error: 0.1581
Epoch 366/500
55/55 [==============================] - 7s 132ms/step - loss: 0.4773 - acc: 0.7534 - mean_squared_error: 0.1587
Epoch 367/500
55/55 [==============================] - 7s 130ms/step - loss: 0.4758 - acc: 0.7542 - mean_squared_error: 0.1582
Epoch 368/500
55/55 [==============================] - 7s 125ms/step - loss: 0.4745 - acc: 0.7554 - mean_squared_error: 0.1576
Epoch 369/500
55/55 [==============================] - 6s 114ms/step - loss: 0.4748 - acc: 0.7549 - mean_squared_error: 0.1577
Epoch 370/500
55/55 [==============================] - 7s 122ms/step - loss: 0.4737 - acc: 0.7565 - mean_squared_error: 0.157

Epoch 427/500
55/55 [==============================] - 7s 135ms/step - loss: 0.4607 - acc: 0.7659 - mean_squared_error: 0.1524
Epoch 428/500
55/55 [==============================] - 7s 122ms/step - loss: 0.4633 - acc: 0.7638 - mean_squared_error: 0.1533
Epoch 429/500
55/55 [==============================] - 6s 114ms/step - loss: 0.4632 - acc: 0.7642 - mean_squared_error: 0.1533
Epoch 430/500
55/55 [==============================] - 7s 123ms/step - loss: 0.4642 - acc: 0.7629 - mean_squared_error: 0.1538
Epoch 431/500
55/55 [==============================] - 6s 115ms/step - loss: 0.4645 - acc: 0.7623 - mean_squared_error: 0.1539
Epoch 432/500
55/55 [==============================] - 7s 128ms/step - loss: 0.4635 - acc: 0.7638 - mean_squared_error: 0.1534
Epoch 433/500
55/55 [==============================] - 7s 126ms/step - loss: 0.4607 - acc: 0.7657 - mean_squared_error: 0.1524
Epoch 434/500
55/55 [==============================] - 7s 120ms/step - loss: 0.4618 - acc: 0.7651 - mean_square

55/55 [==============================] - 7s 123ms/step - loss: 0.4555 - acc: 0.7708 - mean_squared_error: 0.1504
Epoch 492/500
55/55 [==============================] - 7s 127ms/step - loss: 0.4549 - acc: 0.7706 - mean_squared_error: 0.1502
Epoch 493/500
55/55 [==============================] - 7s 120ms/step - loss: 0.4588 - acc: 0.7674 - mean_squared_error: 0.1517
Epoch 494/500
55/55 [==============================] - 7s 129ms/step - loss: 0.4608 - acc: 0.7664 - mean_squared_error: 0.1525
Epoch 495/500
55/55 [==============================] - 7s 133ms/step - loss: 0.4577 - acc: 0.7692 - mean_squared_error: 0.1513
Epoch 496/500
55/55 [==============================] - 6s 113ms/step - loss: 0.4523 - acc: 0.7739 - mean_squared_error: 0.1491
Epoch 497/500
55/55 [==============================] - 6s 118ms/step - loss: 0.4516 - acc: 0.7740 - mean_squared_error: 0.1488
Epoch 498/500
55/55 [==============================] - 7s 119ms/step - loss: 0.4479 - acc: 0.7762 - mean_squared_error: 0.147

level 3
[[44085     0]
 [ 1515     0]] 0.97 0.5
[[44169     0]
 [ 1431     0]] 0.97 0.5
[[44290     0]
 [ 1310     0]] 0.97 0.5
[[44124     0]
 [ 1476     0]] 0.97 0.5
level 4
[[43962     0]
 [ 1638     0]] 0.96 0.5
[[44020     0]
 [ 1580     0]] 0.97 0.5
[[44201     0]
 [ 1399     0]] 0.97 0.5
[[43948     0]
 [ 1652     0]] 0.96 0.5
level 5
[[43733     0]
 [ 1867     0]] 0.96 0.5
[[43862     0]
 [ 1738     0]] 0.96 0.5
[[43917     0]
 [ 1683     0]] 0.96 0.5
[[43744     0]
 [ 1856     0]] 0.96 0.5
level 6
[[43400     0]
 [ 2200     0]] 0.95 0.5
[[43540     0]
 [ 2060     0]] 0.95 0.5
[[43599     0]
 [ 2001     0]] 0.96 0.5
[[43380     0]
 [ 2220     0]] 0.95 0.5
level 7
[[43045     0]
 [ 2555     0]] 0.94 0.5
[[43095     0]
 [ 2505     0]] 0.95 0.5
[[43072     0]
 [ 2528     0]] 0.94 0.5
[[42942     0]
 [ 2658     0]] 0.94 0.5
level 8
[[42677     0]
 [ 2923     0]] 0.94 0.5
[[42588     0]
 [ 3012     0]] 0.93 0.5
[[42491     0]
 [ 3109     0]] 0.93 0.5
[[42348     0]
 [ 3252     0]] 0